In [1]:
!pip install -q transformers accelerate pytest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.5 MB/s eta 0:00:00


In [6]:
import pandas as pd
from IPython.display import display

In [8]:
# Chosen Starcoder2-3B, which won't exceed the RAM limit of the free-tier colab
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, subprocess, tempfile, pathlib

MODEL_NAME = "bigcode/starcoder2-3b"
device     = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

In [15]:
# For now, only used a easy coding question
PROMPT_TEMPLATE = """
You are a senior Python instructor. For the given LeetCode problem, do the following:
1. First output a clear, numbered step-by-step plan.
2. Then write the **final Python function** ONLY between
   the tags:  ### BEGIN SOLUTION  ...  ### END SOLUTION
3. After the END tag, give a one-line time/space complexity note.

Problem:
“Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
Example: nums = [2,7,11,15], target = 9 → [0,1]”
"""


In [10]:
# code generater and the test suite (implemented using pytest)
def generate_code(prompt: str, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def run_pytest(code: str, test_code: str):
    tmp = pathlib.Path(tempfile.mkdtemp())
    (tmp/"solution.py").write_text(code)
    (tmp/"test_solution.py").write_text(test_code)
    proc = subprocess.run(
        ["pytest", "-q"], cwd=tmp, capture_output=True, text=True
    )
    return proc.returncode == 0, proc.stdout + proc.stderr

In [17]:
import re, textwrap

SMART_QUOTES = { "“": '"', "”": '"', "‘": "'", "’": "'" }

def extract_code(raw: str) -> str:
    # tag finding
    m = re.search(r"### BEGIN SOLUTION(.*?)### END SOLUTION",
                  raw, flags=re.S)
    if m:
        code = m.group(1)
    # fallback
    elif "```python" in raw:
        code = raw.split("```python", 1)[1].rsplit("```", 1)[0]
    else:
        code = raw

    for bad, good in SMART_QUOTES.items():
        code = code.replace(bad, good)

    return textwrap.dedent(code).strip()

In [19]:
# automated feed-back loop
TEST_SNIPPET = """
import pytest
from solution import two_sum

def test_example():
    assert two_sum([2,7,11,15], 9) == [0,1]
"""

def solve_two_sum(max_iters=10):
    prompt = PROMPT_TEMPLATE
    iteration_logs = []
    for i in range(max_iters):
        # generate
        raw = generate_code(prompt)
        # extract
        code = extract_code(raw)
        #  (
        #     raw.split("```python")[1].rsplit("```", 1)[0].strip()
        #     if "```python" in raw
        #     else raw.strip()
        # )

        # test
        passed, log = run_pytest(code, TEST_SNIPPET)

        # log
        iteration_logs.append({
            "iter": i + 1,
            "prompt": prompt,
            "llm_output": raw,
            "extracted_code": code,
            "passed": passed,
            "test_log": log
        })

        if passed:
            print(f"Passed after {i+1} iteration(s)\n")
            print(code)
            # return
            break

        log_block = (
            "\nYour previous solution failed with the following error:\n"
            "```\n"
            + log
            + "\n```\n"
        )
        # new prompt
        prompt = (
            log_block
            + "Briefly explain the bug, then provide only the corrected Python function implementation for two_sum.\n"
        )

    else:
        print("Failed after feedback loops. Final attempt:\n")


    df = pd.DataFrame(iteration_logs)
    display(df)



    print(code)

solve_two_sum()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Passed after 6 iteration(s)

def two_sum(nums, target):
    hash_table = {}

    for i in range(len(nums)):
        if hash_table.get(nums[i])!= None:
            return [hash_table[nums[i]], i]
        hash_table[target - nums[i]] = i
    return None


,iter,prompt,llm_output,extracted_code,passed,test_log
0,1,\nYou are a senior Python instructor. For the ...,\nYou are a senior Python instructor. For the ...,...,False,\n==================================== ERRORS ...
1,2,\nYour previous solution failed with the follo...,\nYour previous solution failed with the follo...,"def two_sum(nums, target):\n for i in range...",False,\n==================================== ERRORS ...
2,3,\nYour previous solution failed with the follo...,\nYour previous solution failed with the follo...,Your previous solution failed with the followi...,False,\n==================================== ERRORS ...
3,4,\nYour previous solution failed with the follo...,\nYour previous solution failed with the follo...,"def two_sum(nums, target):\n """"""\n :type...",False,\n==================================== ERRORS ...
4,5,\nYour previous solution failed with the follo...,\nYour previous solution failed with the follo...,Your previous solution failed with the followi...,False,\n==================================== ERRORS ...
5,6,\nYour previous solution failed with the follo...,\nYour previous solution failed with the follo...,"def two_sum(nums, target):\n hash_table = {...",True,. ...


def two_sum(nums, target):
    hash_table = {}

    for i in range(len(nums)):
        if hash_table.get(nums[i])!= None:
            return [hash_table[nums[i]], i]
        hash_table[target - nums[i]] = i
    return None
